In [1]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


In [2]:
#arrestdata json file
dfa = pd.read_json('./data/arrestData.json')

#player info by year
dfr = pd.read_csv('./data/regular_season_rosters.csv')



In [3]:
def crimeCategoryCount(i, category):
    if category:
        dfa.set_value(i, category, 1)
        
[crimeCategoryCount(x, y) for x, y in dfa['Crime_category'].iteritems()]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

['ArrestSeasonState',
 'Category',
 'Crime_category',
 'Crime_category_color',
 'Date',
 'Day',
 'Day_of_Week',
 'Day_of_Week_int',
 'DaysSince',
 'DaysToLastArrest',
 'DaysToLastCrimeArrest',
 'DaysToLastTeamArrest',
 'Description',
 'Encounter',
 'Month',
 'Name',
 'Outcome',
 'Position',
 'Position_name',
 'Position_type',
 'Season',
 'Team',
 'Team_Conference',
 'Team_Conference_Division',
 'Team_Division',
 'Team_city',
 'Team_hex_alt_color',
 'Team_hex_color',
 'Team_logo_id',
 'Team_name',
 'Team_preffered_name',
 'Year',
 'arrest_stats_id',
 'general_category_id',
 'legal_level_id',
 'resolution_category_id',
 'DUI',
 'Other',
 'Drugs',
 'Domestic Violence',
 'Resisting',
 'Assault / Battery',
 'Gun',
 'Alcohol',
 'License / Traffic',
 'Disorderly conduct',
 'Sex',
 'Theft / Burglary',
 'Murder / Manslaughter',
 'Animal Abuse']

In [5]:

dfr.columns = ['No','name','Age','Pos','G','GS','Wt','Ht','College','BirthDate','Yrs','AV','Drafted (tm/rnd/yr)','team','year']

#Seperate information about draft pick into seperate columns
dfr[['Draft_Team', 'Draft_Round', 'Draft_Pick','Draft_Year']] = dfr['Drafted (tm/rnd/yr)'].str.split('/', expand=True)
dfr = dfr.drop('Drafted (tm/rnd/yr)', axis=1)

dfr['team'] = dfr['team'].str.upper()
dfr['Pos'] = dfr['Pos'].str.upper()
dfr['name'] = dfr['name'].str.replace(r'[/*+]','')
dfr['Draft_Pick'] = dfr["Draft_Pick"].str.replace(r'[^0-9]','')

#New(from arrest data) to Old(From pro football reference)
teamMap = { 'ARI':'CRD', 
           'NE':'NWE', 
           'BAL':'RAV', 
           'LAC':'SDG', 
           'NO':'NOR', 
           'SF':'SFO', 
           'GB':'GNB',
           'OAK':'RAI',
           'LA':'RAM',
           'HOU':'HTX', 
           'IND':'CLT', 
           'KC':'KAN',
           'TEN':'OTI',
           'JAC': 'JAX',
           'TB':'TAM'
          }

def teamNameReplace(new, old):
    dfr['team'] = dfr['team'].replace(old,new)
    

{teamNameReplace(new, old) for new, old in teamMap.items()}





{None}

In [6]:
grouped = dfa.groupby(['Name', 'Year', 'Team'])#['Category'].count()
test = grouped.agg({'Category' : 'count', 'DUI' : 'count','Other' : 'count','Drugs':'count', 'Domestic Violence':'count','Resisting':'count','Assault / Battery':'count','Gun':'count','Alcohol':'count','License / Traffic':'count','Disorderly conduct':'count','Sex':'count','Theft / Burglary':'count','Murder / Manslaughter':'count','Animal Abuse':'count'})
print(type(test))
arrestCountByYear = pd.DataFrame(test).reset_index()

####Example Lookup######
#arrestCountByYear[(arrestCountByYear.Name == "David Boston")]
#arrestCountByYear.sample(50)


dfm = pd.merge(dfr, arrestCountByYear, how="outer", left_on=['name', 'year', 'team'], right_on=['Name', 'Year', 'Team'])

#Select players who were arrested and not matched with a regular season roster but still active between 2000-2013.
#aka don't have profile info
dfMissing = dfm[(dfm.Name.notnull()) & (dfm.name.isnull())]

#Shows the remaining players that have profile info
dfFull = dfm[(dfm.name.notnull())]

dfm.to_csv('groupermerge.csv')

#####testing accuracy######## 
#print(len(dfFull) + len(dfMissing))
#print(len(dfm))



<class 'pandas.core.frame.DataFrame'>


In [7]:
len(arrestCountByYear)

761

## Replace Names
Most of the players below can be matched on name to data that appears above.
Players that could not be matched were the result of:
- Stemming issues (ie. Joe to Joseph)
- They did not play on a regular season roster during the period of 2000-2013

In [8]:
dfm['name'] = dfm['name'].str.replace("'","")

def fillBlankProfiles(i,playerRow):
    #print(playerRow[['Name','Year','Team']])
    #print("Found matches in merged dataframe")
 
    lookupDf = dfm[dfm.name.str.upper() == playerRow['Name'].upper()].reset_index()
    
    ##For those whose names appear in the origal dataframe pick the closest year and fill with that data.
    if not(lookupDf.empty):
        x = (abs(lookupDf['year'] - playerRow['Year'])).idxmin()
        dfMissing.set_value(i, 'name', lookupDf.iloc[x]['name'])
        ##age will be calculated
        dfMissing.set_value(i, 'Pos', lookupDf.iloc[x]['Pos'])
        dfMissing.set_value(i, 'G', lookupDf.iloc[x]['G'])
        dfMissing.set_value(i, 'GS', lookupDf.iloc[x]['GS'])
        dfMissing.set_value(i, 'Wt', lookupDf.iloc[x]['Wt'])
        dfMissing.set_value(i, 'Ht', lookupDf.iloc[x]['Ht'])
        dfMissing.set_value(i, 'College', lookupDf.iloc[x]['College'])
        dfMissing.set_value(i, 'BirthDate', lookupDf.iloc[x]['BirthDate'])
        dfMissing.set_value(i, 'Yrs', lookupDf.iloc[x]['Yrs'])
        dfMissing.set_value(i, 'AV', lookupDf.iloc[x]['AV'])
        dfMissing.set_value(i, 'team', playerRow['Team'])
        dfMissing.set_value(i, 'year', playerRow['Year'])
        dfMissing.set_value(i, 'Draft_team', lookupDf.iloc[x]['Draft_Team'])
        dfMissing.set_value(i, 'Draft_Round', lookupDf.iloc[x]['Draft_Round'])
        dfMissing.set_value(i, 'Draft_Pick', lookupDf.iloc[x]['Draft_Pick'])
        dfMissing.set_value(i, 'Draft_Year', lookupDf.iloc[x]['Draft_Year'])

            
            
            
#         print (lookupDf.iloc[0]['name'])
#         dfMissing.set_value(i,'name', lookupDf.iloc[0]['name'])
        
    
[fillBlankProfiles(i, playerRow) for i,playerRow in dfMissing.iterrows()]

dfMissing.to_csv('FillerOut.csv')

/home/brady/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/brady/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
dfx = dfMissing[dfMissing.name.notnull()]
dfFinal = dfFull.append(dfx)
dfFinal.to_csv('./data/final.csv')

list(dfFinal)


7866